In [8]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tqdm
import os
import torch


In [19]:
class_ids_map = {
    '02691156': 'Airplane',
    '02828884': 'Bench',
    '02933112': 'Cabinet',
    '02958343': 'Car',
    '03001627': 'Chair',
    '03211117': 'Display',
    '03636649': 'Lamp',
    '03691459': 'Loudspeaker',
    '04090263': 'Rifle',
    '04256520': 'Sofa',
    '04379243': 'Table',
    '04401088': 'Telephone',
    '04530566': 'Watercraft',
    '1':'',
}
class ShapeNet(object):
    def __init__(self, directory=None, class_ids=None, set_name=None):
        print(class_ids)
        self.class_ids = class_ids
        self.set_name = set_name
        self.elevation = 30.
        self.distance = 2.732

        self.class_ids_map = class_ids_map

        images = []
        voxels = []
        self.num_data = {}
        self.pos = {}
        count = 0
        loop = tqdm.tqdm(self.class_ids)
        #loop = self.class_ids
        loop.set_description('Loading dataset')
        for class_id in loop:
            images.append(list(np.load(
                os.path.join(directory, '%s_%s_images.npz' % (class_id, set_name))).items())[0][1])
            voxels.append(list(np.load(
                os.path.join(directory, '%s_%s_voxels.npz' % (class_id, set_name))).items())[0][1])
            self.num_data[class_id] = images[-1].shape[0]
            self.pos[class_id] = count
            count += self.num_data[class_id]
        images = np.concatenate(images, axis=0).reshape((-1, 4, 64, 64))
        images = np.ascontiguousarray(images)
        print(images.shape)
        self.images = images
        self.voxels = np.ascontiguousarray(np.concatenate(voxels, axis=0))
        print(len(voxels[0]))
        del images
        del voxels

    @property
    def class_ids_pair(self):
        class_names = [self.class_ids_map[i] for i in self.class_ids]
        return zip(self.class_ids, class_names)
    def get_all_batches_for_evaluation(self, batch_size, class_id):
        data_ids = np.arange(self.num_data[class_id]) + self.pos[class_id]
        viewpoint_ids = np.tile(np.arange(24), data_ids.size)
        data_ids = np.repeat(data_ids, 24) * 24 + viewpoint_ids
        print(data_ids)
        for i in range((data_ids.size - 1) // batch_size + 1):
            
            images = torch.from_numpy(
                self.images[data_ids[i * batch_size:(i + 1) * batch_size]].astype('float32') / 255.)
            voxels = torch.from_numpy(
                self.voxels[data_ids[i * batch_size:(i + 1) * batch_size] // 24].astype('float32'))
            yield images, voxels

In [5]:
dataset_test = ShapeNet(r'C:\workspace\SoftRas\examples\recon', {'1_1': ''}, 'val')

Loading dataset: 100%|██████████| 1/1 [00:00<00:00, 393.57it/s]

(1, 4, 64, 64)
1


In [17]:
dataset_val = ShapeNet(r'C:\workspace\SoftRas\examples\recon', '1', 'val')

Loading dataset: 100%|██████████| 1/1 [00:00<00:00, 199.77it/s]

1
(1, 4, 64, 64)
1


In [20]:
for class_id, class_name in dataset_val.class_ids_pair:
    for i, (im, vx) in enumerate(dataset_val.get_all_batches_for_evaluation(1, class_id)):
        images = torch.autograd.Variable(im).cuda()
        voxels = vx.numpy()

AssertionError: Torch not compiled with CUDA enabled